In [1]:
# Import packages
import numpy as np
import pandas as pd
import datetime

import warnings
warnings.filterwarnings("ignore")

# Coordinates

In [2]:
# Load data
data = pd.read_excel('../data/raw/groningen/boreholes.xlsx')
print(data.shape)

(6624, 61)


In [3]:
# Take Groningen
df = data[data['Provincie Naam'] == 'Groningen']
df.reset_index(drop=True, inplace=True)
print(df.shape)

# Pick columns
columns_to_pick = [
    'Boorgatnaam',
    'X TM5_ED50',
    'Y TM5_ED50',
]
df = df[columns_to_pick]
print(df.shape)

# Rename columns
rename_dict = {
    "Boorgatnaam": "cat", 
    "X TM5_ED50": "x",
    "Y TM5_ED50": "y",
    }
df.rename(columns=rename_dict, inplace=True)

df

(633, 61)
(633, 3)


,cat,x,y
0,ALLARDSOOG-01,587913,5884337
1,ALLARDSOOG-01-SIDETRACK1,587913,5884337
2,AMSWEER-01,627042,5907638
3,AMSWEER-02,627324,5907576
4,AMSWEER-03,627254,5907574
...,...,...,...
628,ZUIDWENDING ZOUT-05,629199,5883327
629,ZUIDWENDING ZOUT-06,629316,5883549
630,ZUIDWENDING ZOUT-07,629446,5883773
631,ZUIDWENDING ZOUT-08,629850,5883986


In [4]:
path = "../data/interim/groningen_coords.csv"

df.to_csv(path, index=False)

# Data

In [5]:
products = ["Gas", "Condensate", "Water"]
years = range(2003, 2023+1)
months = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]

In [6]:
# Show shapes
print("---------Gas------Condensate----Water-----|----Nan's---")

for year in years:

    # read each product data by year
    path1 = "../data/raw/groningen/Gas/wellGasProduced_" + str(year) + ".xlsx"
    path2 = "../data/raw/groningen/Condensate/wellCondensateProduced_" + str(year) + ".xlsx"
    path3 = "../data/raw/groningen/Water/wellWaterProduced_" + str(year) + ".xlsx"
    df1 = pd.read_excel(path1, header=1)
    df2 = pd.read_excel(path2, header=1)
    df3 = pd.read_excel(path3, header=1)
    
    # take only Groningen
    df1 = df1[df1["FIELD"] == "Groningen"]
    df2 = df2[df2["FIELD"] == "Groningen"]
    df3 = df3[df3["FIELD"] == "Groningen"]
    

    print(f"{year}: {df1.shape} - {df2.shape} - {df3.shape} - | - {df1.shape[0] - df1.dropna().shape[0]} - {df2.shape[0] - df2.dropna().shape[0]} - {df3.shape[0] - df3.dropna().shape[0]}")

---------Gas------Condensate----Water-----|----Nan's---
2003: (299, 16) - (299, 16) - (299, 16) - | - 0 - 0 - 0
2004: (297, 16) - (297, 16) - (297, 16) - | - 2 - 2 - 2
2005: (299, 16) - (299, 16) - (299, 16) - | - 3 - 3 - 3
2006: (297, 16) - (297, 16) - (297, 16) - | - 0 - 0 - 0
2007: (297, 16) - (297, 16) - (297, 16) - | - 0 - 0 - 0
2008: (297, 16) - (297, 16) - (297, 16) - | - 19 - 19 - 19
2009: (281, 16) - (281, 16) - (281, 16) - | - 3 - 3 - 3
2010: (259, 16) - (259, 16) - (259, 16) - | - 0 - 0 - 0
2011: (259, 16) - (259, 16) - (259, 16) - | - 0 - 0 - 0
2012: (259, 16) - (259, 16) - (259, 16) - | - 0 - 0 - 0
2013: (261, 16) - (261, 16) - (261, 16) - | - 2 - 2 - 2
2014: (259, 16) - (259, 16) - (259, 16) - | - 0 - 0 - 0
2015: (259, 16) - (259, 16) - (259, 16) - | - 1 - 1 - 1
2016: (258, 16) - (258, 16) - (258, 16) - | - 0 - 0 - 0
2017: (258, 16) - (258, 16) - (258, 16) - | - 0 - 0 - 0
2018: (258, 16) - (258, 16) - (258, 16) - | - 0 - 0 - 0
2019: (258, 16) - (258, 16) - (258, 16) - | -

In [7]:
data_res = pd.DataFrame({"cat": [], "date": []})

for product in  products:
    
    data_product = pd.DataFrame()

    for year in years:
        
        # read product data by year
        path = "../data/raw/groningen/" + product + "/well" + product + "Produced_" + str(year) + ".xlsx"
        df = pd.read_excel(path, header=1)
        
        # Take only Groningen
        df = df[df["FIELD"] == "Groningen"]
        df.reset_index(drop=True, inplace=True)
        
        # Make dates
        for i, month in enumerate(months, start=1):
            data_month = pd.DataFrame()
            
            data_month["cat"] = df["WELL"]
            if i < 10:
                data_month["date"] = datetime.datetime.strptime(str(year) + "0" + str(i) + "01", "%Y%m%d")
            else:
                data_month["date"] = datetime.datetime.strptime(str(year) + str(i) + "01", "%Y%m%d")
            data_month[product.lower()] = df[month]
            
            data_product = pd.concat([data_product, data_month], axis=0)

    data_res = pd.merge(data_res, data_product, on=["cat", "date"], how="outer")

data_res.reset_index(drop=True, inplace=True)
data_res.dropna(inplace=True)

In [8]:
data_res.sort_values(by=["cat", "date"], inplace=True)

data_res

,cat,date,gas,condensate,water
0,'T ZANDT-01,2003-01-01,0.000000,0.000,0.0
299,'T ZANDT-01,2003-02-01,0.000000,0.000,0.0
598,'T ZANDT-01,2003-03-01,0.000000,0.000,0.0
897,'T ZANDT-01,2003-04-01,0.000000,0.000,0.0
1196,'T ZANDT-01,2003-05-01,0.000000,0.000,0.0
...,...,...,...,...,...
63488,ZUIDERVEEN-13,2023-03-01,6035.344886,5.603,68.0
63651,ZUIDERVEEN-13,2023-04-01,0.000000,0.000,0.0
63814,ZUIDERVEEN-13,2023-05-01,0.000000,0.000,0.0
63977,ZUIDERVEEN-13,2023-06-01,0.000000,0.000,0.0


In [9]:
path = "../data/interim/groningen_data.csv"

data_res.to_csv(path, index=False)